# The Chat Format

- We will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

- Load the API key and relevant Python libaries.

In [4]:
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
import langchain_core
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.getenv('GROQ_API_KEY')

In [6]:
# model='llama-3.3-70b-versatile'

llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None
)

In [8]:
def get_completion(prompt, model="llama-3.3-70b-versatile"):
    llm.model_name = model
    return llm.invoke(prompt).content


def get_completion_from_messages(messages, model="llama-3.3-70b-versatile", temperature=0):
    temp_llm = ChatGroq(
        api_key=groq_api_key,
        model=model,
        temperature=temperature,
        max_tokens=None
    )
    return temp_llm.invoke(messages).content

In [10]:
messages = [
    {"role": "system", "content": "You are a customer service assistant."},
    {"role": "user", "content": "Write an apology email for late delivery."}
]

print(get_completion_from_messages(messages, temperature=0.7))

Here is an example of an apology email for late delivery:

Subject: Apology for Late Delivery of Your Order

Dear [Customer Name],

I am writing to apologize for the late delivery of your order, [Order Number], which was supposed to arrive on [Expected Delivery Date]. Unfortunately, due to unforeseen circumstances, your order was delayed and was delivered on [Actual Delivery Date].

We understand that you were looking forward to receiving your order on time, and we regret any inconvenience this may have caused. Please know that we take situations like this very seriously and are taking immediate action to prevent such delays in the future.

We want to assure you that we are committed to providing the best possible service to our customers, and we fell short of our own standards in your case. We are truly sorry for the frustration and disappointment this may have caused.

As a gesture of goodwill, we would like to offer you [optional: a discount on your next order, a refund of the shipp

In [16]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [18]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

Fair sir, thou seekest a jest most fine and bright? 
Then heed my words, and mark this wondrous sight: 
A chicken, with feathers fluffed and pride, 
Did cross the road, with purpose to abide. 
And why, thou askest, did this fowl take flight? 
Forsooth, 'twas to get to the other side, in sooth, to dine on ale and cake, a merry night!


In [20]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa, it's nice to meet you. Is there something I can help you with or would you like to chat for a bit? I'm all ears (or in this case, all text). How's your day going so far?


In [22]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm happy to chat with you. However, I don't have any information about your name. I'm a large language model, I don't have the ability to store or recall personal information about our previous conversations. Each time you interact with me, it's a new conversation, and I don't retain any information from previous chats.

If you'd like to share your name with me, I'd be happy to use it in our conversation. Otherwise, I can simply call you "friend" or use a friendly greeting. How can I assist you today?


In [24]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

That's an easy one, Isa. Your name is Isa! I remember you told me just a moment ago.


# OrderBot

- We can automate the collection of user prompts and assistant responses to build a OrderBot. 
- The OrderBot will take orders at a pizza restaurant. 

In [33]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [35]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [37]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   \
 4) list of sides include size  5)total price '},    
)

#The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  
# 4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Here is a JSON summary of your order:
```
{
  "pizza": [
    {
      "type": "cheese",
      "size": "large",
      "price": 10.95
    }
  ],
  "toppings": [],
  "drinks": [],
  "sides": [
    {
      "type": "fries",
      "size": "regular",
      "price": 4.50
    }
  ],
  "total_price": 15.45
}
```
